In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive/My\ Drive/Colab\ Notebooks
%cd FSI/Mnist/

!pip install python-mnist

# **Base Mnist**

Continuamos o estudo da classificação multiclasse agora com a famosa base Mnist.

A base Mnist é formada por imagens de dígitos manuscritos. Ela possui 60000 imagens de treinamento e 10000 imagens de teste que devem ser classificadas de 0 a 9.

![alt text](https://miro.medium.com/max/1728/0*UWCpKRWuPgimzY2R.png)

Cada imagem da base possui 28x28 pixels, e cada pixel é um valor que pode variar de 0-255, indicando o valor grayscale do pixel na imagem.

Na solução proposta pelo grupo, cada imagem será tratada como um vetor.

![alt_text](https://miro.medium.com/max/940/0*teIkFyoG1mj8Ul-r)


Esse vetor será a estrada de atributos de cada exemplo. Logo cada exemplo da base terá 784 atributos que devem ser classificados em 10 classes.

## **Implementação**

Imports necessários para o código.

In [0]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import sys
import pickle
import time
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from mnist import MNIST

Variavel global

In [0]:
class_names = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


Função que plota o gráfico da matriz de confusão

In [0]:
def plot_confusion_matrix(y_true, y_pred, classes, title):
    # Calcula acurácia
    acc = accuracy_score(y_true, y_pred)
    title = title + " (Acurácia: " + str("{:10.4f}".format(acc)) + ")"

    # Constroi matriz de confusão
    cm = confusion_matrix(y_true, y_pred, classes, normalize='true')
    cm_df = pd.DataFrame(cm, index = classes, columns = classes)
    plt.figure(figsize=(5.5,4))
    sns.heatmap(cm_df, annot=True, cmap="YlGnBu")
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

Função que carrega as imagens de treinamento e transforma em vetor de atributos.

Aqui a função sample foi utilizada para reduzir a quantidade de imagens de treinamento para demonstração em sala.

In [0]:
def load_training():

    # Carrega dataset
    print("Carregando dados de treinamento na memória...")
    t0 = time.time()
    mndata = MNIST('.')
    training_images, training_labels = mndata.load_training()

    # Divide dados de treinamento em atributos e classes
    training_images = pd.DataFrame(training_images)
    training_labels = np.array(training_labels)
    training_images = training_images.sample(100)
    training_labels = training_labels[training_images.index]

    t1 = time.time()
    print("Carregamento das imagens de treinamento demorou ", t1-t0, " segundos")

    return training_images, training_labels


Função que carrega as imagens de teste e transforma em vetor de atributos

In [0]:
# Carrega imagens de teste na memória
def load_test():

    # Carrega dataset
    print("Carregando dados de teste na memória...")
    t0 = time.time()
    mndata = MNIST('.')
    test_images, test_labels = mndata.load_testing()

    # Divide dados de treinamento em atributos e classes
    test_images = pd.DataFrame(test_images)
    test_labels = np.array(test_labels)

    t1 = time.time()
    print("Carregamento das imagens de teste demorou ", t1-t0, " segundos")

    return test_images, test_labels

Função que realiza treinamento

In [0]:
# Faz treinamento
def training(clf, filename):
    training_images,training_labels = load_training()

    # Treina classificador
    t0 = time.time()
    clf.fit(training_images, training_labels)
    t1 = time.time()
    print("O treinamento demorou", t1-t0, " segundos")
    pickle.dump(clf, open((filename+".sav"), 'wb'))

Função que realiza as predições

In [0]:

def test(filename):
    test_images, test_labels = load_test()

    t0 = time.time()
    clf = pickle.load(open((filename+".sav"), 'rb'))
    prediction = clf.predict(test_images)
    t1 = time.time()
    print("O teste demorou", t1-t0, " segundos")

    plot_confusion_matrix(test_labels, prediction, classes=class_names,title='Confusion matrix ')


Função menu de escolha dos classificadores binários e multiclasse

In [0]:
# Menu
def menu(mode, mult_clf_mode, bin_clf_mode):

    # Define o classificador binário
    if(bin_clf_mode == "--svc"):
        bin_clf = svm.SVC(class_weight = 'balanced')
        filename = "svc"
    elif(bin_clf_mode == "--mlp"):
        bin_clf = MLPClassifier()
        filename = "mlp"
    else:
        print("Escolha o terceiro argumento como --svc ou --mlp")
        exit()

    # Define o classificador multiclasse
    if(mult_clf_mode == "--ovr"):
        mult_clf = OneVsRestClassifier(bin_clf, n_jobs = -1)
        filename = "ovr_"+filename
    elif(mult_clf_mode == "--ovo"):
        mult_clf = OneVsOneClassifier(bin_clf, n_jobs = -1)
        filename = "ovo_"+filename
    elif(mult_clf_mode == "--eoc"):
        mult_clf = OutputCodeClassifier(bin_clf,code_size =3.0, n_jobs=-1)
        filename = "eoc_"+filename
    else:
        print("Escolha o segundo argumento como --ovr ou --ovo ou --eoc")
        exit()

    if(mode == "--train"):
        training(mult_clf,filename)
    elif(mode == "--test"):
        test(filename)
    else:
        print("Escolha o primeiro argumento como --train ou --test")
        exit()

## **Resultados**

Essa base possui muitos exemplos de teste, logo não é possível demonstrar a execução do código durante a apresentação. Devido ao tempo de treinamento, também não conseguimos treinar essa base usando o classificador binário MLPClassifier. Apresentaremos os resultados obtidos para o classificador SVC.

Como demonstração, apresentaremos os resultados de um treinamento feito com samples de apenas 100 amostras.

In [0]:
menu("--train", "--ovo", "--svc")
menu("--test", "--ovo", "--svc")

### One-vs-Rest

Tempo de treinamento ≃ 75 min

![alt text](https://drive.google.com/uc?id=1fnk_6zjNZZsYn32Bp5o7CkGcGGXL-Ogh)

### One-vs-one

Tempo de treinamento ≃ 5 min

![alt text](https://drive.google.com/uc?id=10Y2WD7QM4LJSjAJ2FRywUJ4su6jfJEJa)

### Error Output Code

Tempo de treinamento ≃ 5 horas

![alt text](https://drive.google.com/uc?id=1755hijjuXN0xL0rweBqiWW9lBeDUXXcP)